#### 4-train_on_train_data_with_saving_model_and_pipelines
like last notebook here we train on all train(no sample), we save the SecureBERT-NER trained model, we generate piplines for preprocess and transformers called:
1. pipeline_from_predictions_to_labels
2. pipeline_from_organized_data_to_lentokenized_data
3. pipeline_from_input_to_lentokenized_data
that help do more simple and efficients processes

we see the recall, precision and latency on test data, we also build pipeline from input text to prdeict labels with pipeline_from_input_to_lentokenized_data

In [35]:
from ner_transformers_and_utilities.functions import ( 
    prepare_data,
    compute_metrics,
    tokenize_and_map_labels,
    plot_training_logs,
    reverse_tokenization,
    split_text_to_words_in_sentences,
)
# 
from sklearn.metrics import precision_score, recall_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import time
import torch
import json
import re
import os
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset
from transformers import Trainer, TrainingArguments
from ner_transformers_and_utilities.data_processing_transformers import (
    DatasetFromListTransformer,
    MapAndTokenizeTransformer,
    ReverseTokenizationTransformer,
)

In [36]:
train_bool = True

In [37]:
test_file_path = 'datasets/test.txt'
val_file_path = 'datasets/valid.txt'
train_file_path = 'datasets/train.txt'

# make val_raw_data and test_raw_data
with open(test_file_path, "r", encoding="utf-8") as file:
    test_raw_data = file.read()

with open(val_file_path, "r", encoding="utf-8") as file:
    val_raw_data = file.read()

with open(train_file_path, "r", encoding="utf-8") as file:
    train_raw_data = file.read()

In [38]:
# step 1 prepare_data(train_raw_data)
# step 2 Dataset.from_list(train_data)
# step 3 train_dataset.map(tokenize_and_map_labels, batched=True)
# Custom transformer for prepare_data
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

tokenizer = AutoTokenizer.from_pretrained("CyberPeace-Institute/SecureBERT-NER")
model = AutoModelForTokenClassification.from_pretrained("CyberPeace-Institute/SecureBERT-NER")

json_map_tokens_path = 'map_tokens.json'

# Check if the file is empty and print its content
with open(json_map_tokens_path, 'r', encoding='utf-8') as file:
    content = file.read()  # Read the file content as a string
    map_tokens = json.loads(content)  # Convert the string to a dictionary


# Step 2: Update the pipeline to use named functions instead of lambdas
pipeline_from_organized_data_to_lentokenized_data = Pipeline([
    ('prepare_data', FunctionTransformer(prepare_data)),
    ('from_list', DatasetFromListTransformer()),
    ('map_and_tokenize', MapAndTokenizeTransformer(tokenizer=tokenizer, map_tokens=map_tokens)),
])

In [39]:
print("len(train_raw_data)", len(train_raw_data))

len(train_raw_data) 1324605


In [40]:
train_raw_data

'The O\nadmin@338 B-HackOrg\nhas O\nlargely O\ntargeted O\norganizations O\ninvolved O\nin O\nfinancial B-Idus\n, O\neconomic B-Idus\nand O\ntrade B-Idus\npolicy I-Idus\n, O\ntypically O\nusing O\npublicly B-Tool\navailable I-Tool\nRATs I-Tool\nsuch O\nas O\nPoison B-Tool\nIvy I-Tool\n, O\nas O\nwell O\nsome O\nnon-public B-Tool\nbackdoors I-Tool\n. O\n\nThe O\nadmin@338 B-HackOrg\nstarted O\ntargeting O\nHong B-Area\nKong I-Area\nmedia B-Org\ncompanies I-Org\n, O\nprobably O\nin O\nresponse O\nto O\npolitical B-Idus\nand O\neconomic B-Idus\nchallenges O\nin O\nHong B-Area\nKong I-Area\nand O\nChina B-Area\n. O\n\nMultiple O\nChina-based B-Area\ncyber B-HackOrg\nthreat I-HackOrg\ngroups I-HackOrg\nhave O\ntargeted O\ninternational B-Org\nmedia I-Org\norganizations I-Org\nin O\nthe O\npast O\n. O\n\nThe O\nadmin@338 B-HackOrg\nhas O\ntargeted O\ninternational B-Org\nmedia I-Org\norganizations I-Org\nin O\nthe O\npast O\n. O\n\nIn O\nAugust B-Time\n2015 I-Time\n, O\nthe O\nadmin@338 B-Ha

In [41]:
# try to get the lentokenized data
tokenized_train_dataset = pipeline_from_organized_data_to_lentokenized_data.transform(train_raw_data)
tokenized_val_dataset = pipeline_from_organized_data_to_lentokenized_data.transform(val_raw_data)


Map:   0%|          | 0/5251 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

In [19]:
if train_bool:
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=5,
        per_device_train_batch_size=8,
        evaluation_strategy="epoch",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        compute_metrics=compute_metrics,
    )



    # Train the model
    trainer.train()

/Users/guybasson/works_assigments/sleek_ml_engineer/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/3285 [00:00<?, ?it/s]

In [20]:
if train_bool:
    train_logs = trainer.state.log_history  # This contains the history of logs
    print(train_logs)
    plot_training_logs(train_logs)

In [21]:
# creagte folder models if not exists
if train_bool:
    if not os.path.exists('models'):
        os.makedirs('models')

    if not os.path.exists('models/securebert_ner'):
        os.makedirs('models/securebert_ner')
         

    # take all name of models 
    name_models = os.listdir('models/securebert_ner')
    if len(name_models) == 0:
        max_index_of_models = 0
    else:    
        max_index_of_models = max([int(name.split('_')[1]) for name in name_models])
    
    # save the model
    model.save_pretrained(f"models/securebert_ner/model_{max_index_of_models + 1}")

In [22]:
if train_bool == False:
    # take all name of models 
    name_models = os.listdir('models/securebert_ner')
    max_index_of_models = max([int(name.split('_')[1]) for name in name_models])
    model = AutoModelForTokenClassification.from_pretrained(f"models/securebert_ner/model_{max_index_of_models}")

In [23]:
reverse_map_tokens = {v: k for k, v in map_tokens.items()}

In [24]:
def reverse_tokenization(predictions, reverse_map_tokens):
    reversed_labels = []
    
    for pred in predictions:
        # Convert the predicted IDs to labels, filtering out the -1 padding
        labels = [reverse_map_tokens.get(label_id, 'O') for label_id in pred if label_id != -1]
        reversed_labels.append(labels)
    
    return reversed_labels

In [25]:
pipeline_from_predictions_to_labels = Pipeline([
    ('reverse_tokenization', ReverseTokenizationTransformer(reverse_map_tokens)),
])

In [26]:
def reverse_tokenization(predictions, reverse_map_tokens):
    reversed_labels = []
    
    for pred in predictions:
        # Convert the predicted IDs to labels, filtering out the -1 padding
        labels = [reverse_map_tokens.get(label_id, 'O') for label_id in pred if label_id != -1]
        reversed_labels.append(labels)
    
    return reversed_labels    

In [27]:
test_data = prepare_data(test_raw_data)

In [28]:
if train_bool:
    start_time = time.time()
    tokenized_test_dataset = pipeline_from_organized_data_to_lentokenized_data.transform(test_raw_data)
    predictions = trainer.predict(tokenized_test_dataset).predictions.argmax(axis=2)
    predict_labels = pipeline_from_predictions_to_labels.transform(predictions)
    end_time = time.time()
    latency = end_time - start_time
    predictions

In [29]:
if train_bool:
    predictions = trainer.predict(tokenized_test_dataset).predictions.argmax(axis=2)

    # Reverse the tokenization to get the original labels
    reversed_labels = reverse_tokenization(predictions, reverse_map_tokens)
    print(reversed_labels)

In [30]:
# take from test_data the labels

if train_bool:
    all_predictions = []
    all_true_labels = []
    # Example of how to print the results
    for i in range(len(test_data)):
        number_of_tokens = len(test_data[i]['tokens'])
        original_tokens = test_data[i]['tokens'][:number_of_tokens]
        predicted_labels = predict_labels[i][:number_of_tokens]
        real_labels = test_data[i]['labels']
        all_predictions.append(predicted_labels)
        all_true_labels.append(real_labels)
    all_predictions_flat = [label for labels in all_predictions for label in labels]
    all_true_labels_flat = [label for labels in all_true_labels for label in labels]    
    # Calculate precision and recall
    precision = precision_score(all_true_labels_flat, all_predictions_flat, average='weighted', zero_division=True)  # Use 'macro' or 'micro' as needed
    recall = recall_score(all_true_labels_flat, all_predictions_flat, average='weighted', zero_division=True)  # Use 'macro' or 'micro' as needed
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("latentcy:", np.round(latency, 2), "Sec")


In [31]:
input_test_example = '''We observed DDKONG in use between February 2017 and the present, while PLAINTEE is a newer addition with the earliest known sample being observed in October 2017. The RANCOR campaign represents a continued trend of targeted attacks against entities within the South East Asia region.'''

In [32]:
# pipeline from words to tokenizer
pipeline_from_input_to_lentokenized_data = Pipeline([
    ('split_text_to_words_in_sentences', FunctionTransformer(split_text_to_words_in_sentences)),
     ('from_list', DatasetFromListTransformer()),  # Use the named function here
    ('map_and_tokenize',  MapAndTokenizeTransformer(tokenizer=tokenizer, map_tokens=map_tokens)),
    
])

In [33]:
# predict the example :
tokenized_input_example = pipeline_from_input_to_lentokenized_data.transform(input_test_example)
if train_bool:
    predictions = trainer.predict(tokenized_input_example).predictions.argmax(axis=2)
    predict_labels = pipeline_from_predictions_to_labels.transform(predictions)
    print(predict_labels)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [34]:
# check if folder pipelines exists
if not os.path.exists('pipelines'):
    os.makedirs('pipelines')

# save the pipeline_from_predictions_to_labels
with open('pipelines/pipeline_from_predictions_to_labels.pkl', 'wb') as file:
    pickle.dump(pipeline_from_predictions_to_labels, file)

# save the pipeline_from_organized_data_to_lentokenized_data
with open('pipelines/pipeline_from_organized_data_to_lentokenized_data.pkl', 'wb') as file:
    pickle.dump(pipeline_from_organized_data_to_lentokenized_data, file)

# save the pipeline_from_input_to_lentokenized_data
with open('pipelines/pipeline_from_input_to_lentokenized_data.pkl', 'wb') as file:
    pickle.dump(pipeline_from_input_to_lentokenized_data, file)